In [ ]:
# 설치 & 임포트
# !pip install openai rouge-score pandas tqdm

import pandas as pd
import time
from tqdm import tqdm
from rouge_score import rouge_scorer
from openai import OpenAI

In [ ]:
# API 클라이언트 설정
UPSTAGE_API_KEY = "up_EgIQb4QV9VIQT1osO07DJm26jvDCh"

client = OpenAI(
    api_key=UPSTAGE_API_KEY,
    base_url="https://api.upstage.ai/v1/solar"
)

In [ ]:
# 데이터 로드
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

print(f"Train: {len(train_df)}개")
print(f"Test: {len(test_df)}개")

In [ ]:
# 프롬프트 (고정 Few-shot 예시)
def build_prompt(dialogue):
    system_prompt = """당신은 한국어 대화 요약 전문가입니다.

## 요약 규칙
1. 길이: 1~2문장, 40~80자
2. 문체: 존댓말 (~합니다, ~습니다)
3. 화자: 이름이 있으면 이름 사용, 없으면 #Person1#, #Person2# 사용
4. 핵심 행동과 결과만 포함
5. 인사, 감사, 구체적 숫자/시간은 생략"""

    # Few-shot 예시 1
    ex1_dialogue = """#Person1#: 안녕하세요, Mr. Smith. 저는 Dr. Hawkins입니다. 오늘 무슨 일로 오셨어요?
#Person2#: 건강검진을 받으려고 왔어요.
#Person1#: 네, 5년 동안 검진을 안 받으셨네요. 매년 한 번씩 받으셔야 해요.
#Person2#: 알죠. 특별히 아픈 데가 없으면 굳이 갈 필요가 없다고 생각했어요.
#Person1#: 담배 피우세요?
#Person2#: 네.
#Person1#: 담배가 폐암하고 심장병의 주된 원인이에요. 끊으셔야 해요.
#Person2#: 수백 번 시도했는데, 도저히 습관이 안 끊어져요.
#Person1#: 도움 될만한 수업과 약물들이 있습니다."""
    ex1_summary = "Mr. Smith는 Dr. Hawkins에게 건강검진을 받으러 와서, 매년 검진 필요성을 안내받고 흡연 습관 개선을 위한 도움을 제안받았습니다."

    # Few-shot 예시 2
    ex2_dialogue = """#Person1#: 저기요, 열쇠 세트 본 적 있어요?
#Person2#: 어떤 종류의 열쇠요?
#Person1#: 열쇠 다섯 개랑 작은 발 장식이 달려 있어요.
#Person2#: 아, 안타깝네요! 못 봤어요.
#Person1#: 그럼, 같이 좀 찾아주실 수 있어요?
#Person2#: 물론이죠. 도와드릴게요."""
    ex2_summary = "#Person1#은 열쇠 세트를 잃어버리고 #Person2#에게 찾는 것을 도와달라고 요청합니다."

    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"대화:\n{ex1_dialogue}\n\n요약:"},
        {"role": "assistant", "content": ex1_summary},
        {"role": "user", "content": f"대화:\n{ex2_dialogue}\n\n요약:"},
        {"role": "assistant", "content": ex2_summary},
        {"role": "user", "content": f"대화:\n{dialogue}\n\n요약:"},
    ]

In [ ]:
# 요약 함수
def summarization(dialogue):
    try:
        response = client.chat.completions.create(
            model="solar-1-mini-chat",
            messages=build_prompt(dialogue),
            temperature=0.1,
            max_tokens=100,
        )
        result = response.choices[0].message.content.strip()
        return result
    except Exception as e:
        print(f"Error: {e}")
        return ""

In [ ]:
# 테스트
sample = train_df.iloc[0]
pred = summarization(sample['dialogue'])

print("=== 생성 ===")
print(pred)
print(f"길이: {len(pred)}자")
print("\n=== 정답 ===")
print(sample['summary'])
print(f"길이: {len(sample['summary'])}자")

In [ ]:
# ROUGE 평가
def evaluate_sample(df, n=50):
    sample_df = df.sample(n, random_state=42)
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=False)
    
    r1, r2, rL = [], [], []
    results = []
    
    for idx, row in tqdm(sample_df.iterrows(), total=len(sample_df)):
        pred = summarization(row['dialogue'])
        scores = scorer.score(row['summary'], pred)
        
        r1.append(scores['rouge1'].fmeasure)
        r2.append(scores['rouge2'].fmeasure)
        rL.append(scores['rougeL'].fmeasure)
        
        results.append({'pred': pred, 'gold': row['summary']})
        time.sleep(0.5)
    
    print(f"\nROUGE-1: {sum(r1)/len(r1):.4f}")
    print(f"ROUGE-2: {sum(r2)/len(r2):.4f}")
    print(f"ROUGE-L: {sum(rL)/len(rL):.4f}")
    print(f"Total: {sum(r1)/len(r1) + sum(r2)/len(r2) + sum(rL)/len(rL):.4f}")
    
    return pd.DataFrame(results)

results_df = evaluate_sample(train_df, n=50)

In [ ]:
# 결과 비교
for i in range(5):
    row = results_df.iloc[i]
    print(f"=== {i+1} ===")
    print(f"생성: {row['pred']}")
    print(f"정답: {row['gold']}")
    print()

In [ ]:
# 전체 테스트 추론
def inference_test(test_df):
    summaries = []
    
    for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
        pred = summarization(row['dialogue'])
        summaries.append(pred)
        
        if (idx + 1) % 100 == 0:
            time.sleep(5)
    
    output = pd.DataFrame({"fname": test_df['fname'], "summary": summaries})
    output.to_csv("submission_solar.csv", index=False)
    print("저장 완료: submission_solar.csv")
    return output

# 실행
# output = inference_test(test_df)